In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.vector_ar.vecm import coint_johansen, VECM
from statsmodels.tsa.stattools import adfuller

In [49]:
# Laste poll-of-polls data
url = "https://raw.githubusercontent.com/jensmorten/onesixtynine/main/data/pollofpolls_master.csv"
df = pd.read_csv(url)

In [50]:
# Convert to datetime and set the date to the end of the month
df["Mnd"] = pd.to_datetime(df["Mnd"])

In [51]:
# Sort values and set index
df = df.sort_values("Mnd")
df.set_index("Mnd", inplace=True)
df.index.to_period('M').to_timestamp('M')

DatetimeIndex(['2008-01-31', '2008-02-29', '2008-03-31', '2008-04-30',
               '2008-05-31', '2008-06-30', '2008-07-31', '2008-08-31',
               '2008-09-30', '2008-10-31',
               ...
               '2025-01-31', '2025-02-28', '2025-03-31', '2025-04-30',
               '2025-05-31', '2025-06-30', '2025-07-31', '2025-08-31',
               '2025-09-30', '2025-10-31'],
              dtype='datetime64[ns]', name='Mnd', length=214, freq='ME')

In [52]:
print(df.index[-5:])  # check last few dates

DatetimeIndex(['2025-06-30', '2025-07-31', '2025-08-31', '2025-09-30',
               '2025-10-31'],
              dtype='datetime64[ns]', name='Mnd', freq=None)


In [53]:
df=df[['Ap', 'Hoyre', 'Frp', 'SV', 'SP', 'KrF', 'Venstre', 'MDG', 'Rodt','Andre']]
df.dropna(inplace=True)

In [54]:
lags=3

In [55]:
n_months=12

In [56]:
adf_results = {}
for col in df.columns:
    res = adfuller(df[col].values)
    adf_results[col] = {'adf_stat': res[0], 'pvalue': res[1]}
pd.DataFrame(adf_results).T

,adf_stat,pvalue
Ap,-1.885138,0.339084
Hoyre,-2.014064,0.280441
Frp,-1.696715,0.432806
SV,-1.499619,0.533700
SP,-1.615324,0.475181
KrF,-2.365845,0.151609
Venstre,-2.915754,0.043546
MDG,-1.487138,0.539864
Rodt,-0.486862,0.894555
Andre,-1.582213,0.492522


In [57]:
#model = VAR(endog=df)
model = VECM(df, k_ar_diff=2, coint_rank=1, deterministic="ci")

C:\Users\jens.nilsen\python\WPy64-31230\python-3.12.3.amd64\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency ME will be used.
  self._init_dates(dates, freq)


In [58]:
#model_fitted = model.fit(maxlags=lags,method = 'ols', trend='n', verbose=True)
vecm_fitted = model.fit()

In [62]:
print(statsmodels.__version__)

0.14.1


In [64]:
var_from_vecm = vecm_fitted._var

AttributeError: 'VECMResults' object has no attribute '_var'

In [ ]:
forecast, forecast_lower, forecast_upper = model_fitted.forecast_interval(model_fitted.endog, steps=n_months)

# --- Create DataFrames for forecast ---
last_date = df.index[-1]
forecast_index = pd.date_range(start=last_date, periods=n_months + 1, freq="M")[1:]

forecast_df = pd.DataFrame(forecast, index=forecast_index, columns=df.columns)
forecast_lower_df = pd.DataFrame(forecast_lower, index=forecast_index, columns=df.columns)
forecast_upper_df = pd.DataFrame(forecast_upper, index=forecast_index, columns=df.columns)


In [ ]:
forecast_df

In [ ]:
# --- Define colors for consistency ---
colors = {
    'Ap': '#FF0000',        # Red
    'Hoyre': '#0000FF',     # Blue
    'Frp': '#00008B',       # Dark Blue
    'SV': '#FF6347',        # Light Red (Tomato)
    'SP': '#006400',        # Dark Green
    'KrF': '#FFD700',       # Yellow (Gold)
    'Venstre': '#ADD8E6',   # Light Blue
    'MDG': '#008000',       # Green
    'Rodt': '#8B0000',      # Dark Red
    'Andre': '#808080'      # Gray
}

# --- Plot ---
plt.figure(figsize=(14, 7))

months_back = 12
df_recent = df.iloc[-months_back:]  # show only last 12 months of actual data

for party, color in colors.items():
    # Plot actual data (last 12 months)
    plt.plot(df_recent.index, df_recent[party], marker="o", color=color, label=f"{party}")

    # Plot forecasted data
    plt.plot(forecast_df.index, forecast_df[party], linestyle="dashed", color=color)

    # Connect actual to first forecast point (smooth transition)
    last_actual_date = df_recent.index[-1]
    first_forecast_date = forecast_df.index[0]
    last_actual_value = df_recent[party].iloc[-1]
    first_forecast_value = forecast_df[party].iloc[0]

    plt.plot([last_actual_date, first_forecast_date],
             [last_actual_value, first_forecast_value],
             color=color, linestyle="dashed")

    # Plot confidence intervals
    #plt.fill_between(
    #    forecast_df.index,
    #    forecast_lower_df[party],
    #    forecast_upper_df[party],
    #    color=color,
    #    alpha=0.15
    #)

# --- Vertical and horizontal guides ---
dates = pd.date_range(start=df_recent.index[0], end=forecast_df.index[-1], freq="MS")
for date in dates:
    plt.axvline(date, color="gray", linestyle="dotted", alpha=0.3)

for percent in range(0, 45, 5):
    plt.axhline(percent, color="gray", linestyle="dotted", alpha=0.3)

# --- Final formatting ---
plt.xlim(df_recent.index[0], forecast_df.index[-1])
plt.ylim(0, 40)
plt.xlabel("Tid")
plt.ylabel("Prosent oppslutning")
plt.title("OneSixtyNine v2.0: Partienes oppslutning (12 mnd historikk + 6 mnd prognose)")
plt.legend(loc="upper left", ncol=2)
plt.grid(alpha=0.2)
plt.tight_layout()
plt.show()